In [ ]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 23.0 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import DiscreteBayesianNetwork as  DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD as TabularCPD
from pgmpy.inference import VariableElimination as VariableElimination


In [ ]:
model_structure = DiscreteBayesianNetwork([
   ( 'Burglary','Alarm'),
   ('Earthquake','Alarm'),
   ('Alarm','JohnCalls'),
   ('Alarm','MaryCalls')
])

In [ ]:
cpd_burglary = TabularCPD(variable='Burglary',variable_card=2,values=[[.999],[.001]])
cpd_earthquake = TabularCPD(variable='Earthquake',variable_card=2,values=[[.998],[.002]])


In [ ]:
print(cpd_burglary)

+-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+


In [ ]:
print(cpd_earthquake)

+---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+


In [ ]:
cpd_alarm = TabularCPD(variable= 'Alarm',variable_card=2,
                       values=[[0.999,0.71,0.06,0.05] ,
                       [0.001,0.29,0.94,0.95]],
                       evidence=['Burglary','Earthquake'],
                       evidence_card=[2,2])
print(cpd_alarm)


+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+


In [ ]:
cpd_johncalls = TabularCPD(variable= 'JohnCalls',variable_card=2,
                       values=[[0.95,0.1],[0.05,0.9]],
                       evidence=['Alarm'],
                       evidence_card=[2])
print(cpd_johncalls)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| JohnCalls(0) | 0.95     | 0.1      |
+--------------+----------+----------+
| JohnCalls(1) | 0.05     | 0.9      |
+--------------+----------+----------+


In [ ]:
cpd_marycalls = TabularCPD(variable= 'MaryCalls',variable_card=2,
                       values=[[0.99,0.3],[0.01,0.7]],
                       evidence=['Alarm'],
                       evidence_card=[2])
print(cpd_marycalls)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| MaryCalls(0) | 0.99     | 0.3      |
+--------------+----------+----------+
| MaryCalls(1) | 0.01     | 0.7      |
+--------------+----------+----------+


In [ ]:
model_structure.add_cpds(cpd_burglary,cpd_earthquake,cpd_alarm,cpd_johncalls,cpd_marycalls)
model_structure.check_model()

True

In [ ]:
inference = VariableElimination(model_structure)
query_result = inference.query(variables=['Burglary'],evidence={'JohnCalls':1,'MaryCalls':1})
print(query_result)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
#query 1
evidence ={
    'Burglary' :0,
    'Earthquake' : 0,

}
result1 = inference.query(variables=['JohnCalls','MaryCalls','Alarm'],evidence=evidence)
print(result1)

+--------------+--------------+----------+----------------------------------+
| JohnCalls    | MaryCalls    | Alarm    |   phi(JohnCalls,MaryCalls,Alarm) |
+==============+==============+==========+==================================+
| JohnCalls(0) | MaryCalls(0) | Alarm(0) |                           0.9396 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(0) | Alarm(1) |                           0.0000 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(1) | Alarm(0) |                           0.0095 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(1) | Alarm(1) |                           0.0001 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(1) | MaryCalls(0) | Alarm(0) |                           0.0495 |
+--------------+--------------+----------+----------------------

In [ ]:
probabailty = result1.values[1][1][1]
print(probabailty)


0.0006300000000000001


In [ ]:
print("probalbity of alarm ringing when johncall= TRUE,marycall= true,burglary=false,earthquake=false")
print(probabailty)

probalbity of alarm ringing when johncall= TRUE,marycall= true,burglary=false,earthquake=false
0.0006300000000000001


In [ ]:
#query2
result2 = inference.query(variables=['JohnCalls'])
print(result2)

+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9479 |
+--------------+------------------+
| JohnCalls(1) |           0.0521 |
+--------------+------------------+


In [ ]:
probabailty2 = result2.values[1]
print(probabailty2)
print("probablity of johncall ",probabailty2)

0.0521389757
probablity of johncall  0.0521389757


In [ ]:
#query3
evidence={  'JohnCalls':1,'MaryCalls':1}
result3 = inference.query(variables=['Burglary'],evidence=evidence)
print(result3)


+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
probabailty3=result3.values[1]
print(probabailty3)
print("prob of when johncall n mary call both 1:",probabailty3)

0.284171835364393
prob of when johncall n mary call both 1: 0.284171835364393


In [ ]:
result4 = inference.query(variables=['Burglary','Earthquake'],evidence={'JohnCalls':1,'MaryCalls':1})
print(result4)

+-------------+---------------+----------------------------+
| Burglary    | Earthquake    |   phi(Burglary,Earthquake) |
+=============+===============+============================+
| Burglary(0) | Earthquake(0) |                     0.5403 |
+-------------+---------------+----------------------------+
| Burglary(0) | Earthquake(1) |                     0.1755 |
+-------------+---------------+----------------------------+
| Burglary(1) | Earthquake(0) |                     0.2836 |
+-------------+---------------+----------------------------+
| Burglary(1) | Earthquake(1) |                     0.0006 |
+-------------+---------------+----------------------------+


In [ ]:
probabailty4=result4.values[0][0]
print(probabailty4)
print("prob of burglar[1]and earthquake 0 and evidence: ",probabailty4)

0.5403356987955319
prob of burglar[1]and earthquake 0 and evidence:  0.5403356987955319


In [ ]:
result5=inference.query(variables=['Burglary','Earthquake'])
print(result5)

+-------------+---------------+----------------------------+
| Burglary    | Earthquake    |   phi(Burglary,Earthquake) |
+=============+===============+============================+
| Burglary(0) | Earthquake(0) |                     0.9970 |
+-------------+---------------+----------------------------+
| Burglary(0) | Earthquake(1) |                     0.0020 |
+-------------+---------------+----------------------------+
| Burglary(1) | Earthquake(0) |                     0.0010 |
+-------------+---------------+----------------------------+
| Burglary(1) | Earthquake(1) |                     0.0000 |
+-------------+---------------+----------------------------+


In [ ]:
probabailty5=result5.values[0][1]
print(probabailty5)
print("prob of burglary[0] and earthquake[1]:",probabailty5)

0.001998
prob of burglary[0] and earthquake[1]: 0.001998


In [ ]:
#query6
evidence={'MaryCalls':1}
result6 = inference.query(variables=['Earthquake'],evidence=evidence)
print(result6)

+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9641 |
+---------------+-------------------+
| Earthquake(1) |            0.0359 |
+---------------+-------------------+


In [ ]:
#query7
result7 = inference.query(variables=['MaryCalls'])
print(result7)

+--------------+------------------+
| MaryCalls    |   phi(MaryCalls) |
+==============+==================+
| MaryCalls(0) |           0.9883 |
+--------------+------------------+
| MaryCalls(1) |           0.0117 |
+--------------+------------------+
